In [2]:
from FRED_module import fetch_FRED
from EUROSTAT_module import fetch_EUROSTAT
from WB_module import fetch_WB 
from YFINANCE_module import fetch_YFINANCE
from DBNOMICS_module import fetch_DBNOMICS